# Comparison of Custom Transformer and GPT-2 for Arithmetic Prediction

## Overview:
- **Objective**: Evaluate and compare a custom transformer model and a fine-tuned GPT-2 model on arithmetic expressions.
- **Steps**:
  1. **Model Setup**: Loaded custom transformer and GPT-2 models with their respective tokenizers.
  2. **Dataset Preparation**:
     - Loaded arithmetic datasets, tokenized, and split into training, validation, and test sets.
  3. **Evaluation**:
     - Tested both models on the test dataset.
     - Decoded and compared predictions against expected outputs.
  4. **Results**:
     - Accuracy(Exact Match and Similarity Percentage)
     - Error Rate
     - Inference Time Analysis
     - Error Analysis


## Setup: Import Libraries and Set Device

- **Libraries**:
  - `torch`: For PyTorch operations.
  - `transformers`: For GPT-2 model and tokenizer.

- **Device Check**:
  - Determines if CUDA-compatible GPU is available.


In [8]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Custom Tokenizer for Arithmetic Expressions

- **`ArithmeticTokenizer` Class**:
  - Defines a vocabulary of characters: digits (`0-9`), operators (`+-*/`), parentheses, equals sign (`=`), space, `<PAD>`, and `<MASK>`.
  - `encode`: Converts input text to a list of corresponding token indices.
  - `decode`: Converts token indices back to text, ignoring `<PAD>` tokens.

- **Initialization**:
  - Instantiates the `ArithmeticTokenizer`.


In [9]:
# Define a tokenizer
class ArithmeticTokenizer:
    def __init__(self):
        self.vocab = {char: idx for idx, char in enumerate("0123456789+-*/().= ")}
        self.vocab["<PAD>"] = len(self.vocab)
        self.vocab["<MASK>"] = len(self.vocab)
        self.reverse_vocab = {idx: char for char, idx in self.vocab.items()}

    def encode(self, text):
        return [self.vocab.get(char, self.vocab["<PAD>"]) for char in text]

    def decode(self, indices):
        return "".join([self.reverse_vocab[idx] for idx in indices if idx in self.reverse_vocab and idx != self.vocab["<PAD>"]])

# Instantiate tokenizer
tokenizer = ArithmeticTokenizer()

print("Tokenizer Initialized")

Tokenizer Initialized


## Transformer-Based Model for Arithmetic Expressions

- **`TransformerArithmeticModel` Class**:
  - **Purpose**: Implements a transformer model for processing arithmetic expressions.
  - **Components**:
    1. **Embedding Layer**: Maps token indices to dense vector representations (`d_model` size).
    2. **Transformer**: Core transformer architecture with multi-head attention (`nhead`), layers (`num_layers`), and dropout.
    3. **Output Layer**: Linear layer maps transformer outputs to the vocabulary size for predictions.
    4. **Dropout**: Regularization for embedding and transformer layers.

- **Methods**:
  - `forward(src, tgt)`: Processes source (`src`) and target (`tgt`) sequences for sequence-to-sequence tasks.
  - `pretrain_forward(src)`: Processes only the source sequence, useful for pretraining tasks (e.g., masked language modeling).


In [10]:
import torch.nn as nn
class TransformerArithmeticModel(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=8, num_layers=6, dropout=0.1):
        super(TransformerArithmeticModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.transformer = nn.Transformer(d_model, nhead, num_layers, num_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, tgt):
        src_emb = self.dropout(self.embedding(src))
        tgt_emb = self.dropout(self.embedding(tgt))
        output = self.transformer(src_emb, tgt_emb)
        return self.fc_out(output)

    def pretrain_forward(self, src):
        src_emb = self.dropout(self.embedding(src))
        output = self.transformer(src_emb, src_emb)
        return self.fc_out(output)



## Load Fine-Tuned Arithmetic Transformer Model

- **Model Path**: Loads a pre-trained model from `arithmetic_model_fineTuned_1DS.pth`.
- **Model**:
  - Instantiates `TransformerArithmeticModel` with a vocabulary size matching the custom tokenizer.
  - Moves the model to the appropriate device (`cuda` or `cpu`).
- **State**:
  - Loads saved model weights.
  - Sets the model to evaluation mode (`eval()`).


In [11]:
# Custom Arithmetic LLM fine-tuned model
custom_model_path = "/home/lxp334/LLM_Final_Report/ArithmetiLLM_fineTuned_1DS.pth"
custom_transformer_model = TransformerArithmeticModel(vocab_size=len(tokenizer.vocab)).to(device)
custom_transformer_model.load_state_dict(torch.load(custom_model_path),strict=False)
custom_transformer_model.eval()

/tmp/job.2285984.hpc/ipykernel_1029777/1313751407.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  custom_transformer_model.load_state_dict(torch.load(custom_model_path),

TransformerArithmeticModel(
  (embedding): Embedding(21, 256)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderL

## Load Fine-Tuned GPT-2 Model

- **Tokenizer**:
  - Initialized with the GPT-2 tokenizer.
  - Configured `pad_token` to match `eos_token` for compatibility.

- **Model Path**: Loads a fine-tuned GPT-2 model from `fine_tuned_gpt2.pth`.

- **Model**:
  - Instantiates GPT-2 (`GPT2LMHeadModel`) and loads pre-trained weights.
  - Sets the model to evaluation mode (`eval()`).


In [12]:
# Initialize the GPT-2 tokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

# GPT-2 fine-tuned model
gpt2_model_path = "/home/lxp334/LLM_Final_Report/fine_tuned_gpt2.pth"
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")
gpt2_model.load_state_dict(torch.load(gpt2_model_path), strict=False)
gpt2_model.eval()  # Set model to evaluation mode



/tmp/job.2285984.hpc/ipykernel_1029777/3949272070.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  gpt2_model.load_state_dict(torch.load(gpt2_model_path), strict=False)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## Model Evaluation: Custom Transformer and GPT-2

### Key Steps:
1. **Model Preparation**:
   - Moved `custom_transformer_model` and `gpt2_model` to the specified device (`cuda` or `cpu`).
   - Verified the devices for both models.

2. **Dataset Handling**:
   - **Dataset Loading**: Combined datasets from file paths and tokenized expressions and results.
   - **Dataset Splitting**: Divided data into training (80%), validation (10%), and test (10%).
   - **Custom Dataset Class**: Defined `ArithmeticTestDataset` for PyTorch compatibility.
   - **Collation**: Used `pad_sequence` for batch preparation in `DataLoader`.

3. **Evaluation Function**:
   - **Custom LLM**:
     - Processed input and target sequences with the `custom_transformer_model`.
     - Decoded inputs, targets, and predictions using `ArithmeticTokenizer`.
   - **GPT-2**:
     - Used the raw expression as input and generated predictions.
     - Extracted and decoded predictions from GPT-2 outputs.

4. **Results**:
   - Compared predictions from both models against expected outputs.
   - Displayed and saved the first 15 results and stored all evaluation results in `evaluation_results.txt`.

### Output:
- Tabular display of:
  - Arithmetic expression.
  - Expected result.
  - Predictions from `custom_transformer_model` and `gpt2_model`


In [13]:
custom_transformer_model.to(device)
gpt2_model.to(device)  # Move GPT-2 to the same device as the custom model

print(f"Custom Model Device: {next(custom_transformer_model.parameters()).device}")
print(f"GPT-2 Model Device: {next(gpt2_model.parameters()).device}")


import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split

# Corrected load_multiple_datasets function
def load_multiple_datasets(file_paths, tokenizer):
    """Loads and tokenizes multiple datasets."""
    all_data = []
    for file_path in file_paths:
        print(f"Loading dataset from: {file_path}")
        with open(file_path, 'r') as f:
            lines = f.readlines()
            for i in range(0, len(lines), 2):
                expression = lines[i].strip()
                result = lines[i + 1].strip()
                # Convert to tensors here
                all_data.append((torch.tensor(tokenizer.encode(expression)), torch.tensor(tokenizer.encode(result))))
    print(f"Total samples from all datasets: {len(all_data)}")
    return all_data

# File paths for datasets
file_paths = [
    "/home/lxp334/LLM_Final_Report/arithmetic__mixed.txt"
]

# Load and combine datasets
combined_data = load_multiple_datasets(file_paths, tokenizer)

# Debug: Print dataset stats and a sample
print(f"Total Samples: {len(combined_data)}")
print(f"Sample Data (Encoded): {combined_data[:1]}")

# Split the dataset into training, validation, and test sets
train_data, test_data = train_test_split(combined_data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

# Debug: Print split sizes
print(f"Train Samples: {len(train_data)}")
print(f"Validation Samples: {len(val_data)}")
print(f"Test Samples: {len(test_data)}")

# Dataset class for evaluation
class ArithmeticTestDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Collate function for DataLoader
def collate_fn(batch):
    expressions, results = zip(*batch)
    padded_expressions = pad_sequence(expressions, batch_first=True, padding_value=tokenizer.vocab["<PAD>"])
    padded_results = pad_sequence(results, batch_first=True, padding_value=tokenizer.vocab["<PAD>"])
    return padded_expressions, padded_results

# Create the DataLoader for test data
test_dataset = ArithmeticTestDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

# Unified evaluation function
def evaluate_models(custom_model, gpt2_model, test_loader, arithmetic_tokenizer, gpt2_tokenizer):
    custom_model.eval()
    gpt2_model.eval()
    results = []

    with torch.no_grad():
        for src, tgt in test_loader:
            # Move tensors to the correct device
            src = src.to(device)
            tgt = tgt.to(device)

            # ----- Custom LLM Predictions -----
            custom_output = custom_model(src, tgt[:, :-1])
            custom_predictions = torch.argmax(custom_output, dim=2)

            # Decode inputs, targets, and predictions for Custom LLM
            src_decoded = arithmetic_tokenizer.decode(
                [t.item() for t in src[0] if t.item() != arithmetic_tokenizer.vocab["<PAD>"]]
            )
            tgt_decoded = arithmetic_tokenizer.decode(
                [t.item() for t in tgt[0] if t.item() != arithmetic_tokenizer.vocab["<PAD>"]]
            )
            custom_pred_decoded = arithmetic_tokenizer.decode(
                [t.item() for t in custom_predictions[0] if t.item() != arithmetic_tokenizer.vocab["<PAD>"]]
            )

            # ----- GPT-2 Predictions -----
            # Use raw expression as input for GPT-2
            gpt2_inputs = gpt2_tokenizer(src_decoded, return_tensors="pt", padding=True, truncation=True).to(device)
            gpt2_output = gpt2_model.generate(
                input_ids=gpt2_inputs["input_ids"],
                attention_mask=gpt2_inputs["attention_mask"],
                max_length=50,
                eos_token_id=gpt2_tokenizer.eos_token_id
            )

            # Decode the entire output and extract the result
            gpt2_full_output_decoded = gpt2_tokenizer.decode(gpt2_output[0], skip_special_tokens=True)
            if src_decoded in gpt2_full_output_decoded:
                gpt2_pred_decoded = gpt2_full_output_decoded.replace(src_decoded, "").strip()
            else:
                gpt2_pred_decoded = gpt2_full_output_decoded

            # Collect results
            results.append((src_decoded, tgt_decoded, custom_pred_decoded, gpt2_pred_decoded))

    return results


# Evaluate the models on the entire test dataset
evaluation_results = evaluate_models(
    custom_transformer_model, 
    gpt2_model, 
    test_loader, 
    arithmetic_tokenizer=tokenizer,  # ArithmeticTokenizer for Custom LLM
    gpt2_tokenizer=gpt2_tokenizer    # GPT2Tokenizer for GPT-2
)

# Display results
print(f"{'Expression':<30} | {'Expected':<20} | {'Custom LLM Prediction':<30} | {'GPT-2 Prediction':<30}")
print("-" * 120)
for src, tgt, custom_pred, gpt2_pred in evaluation_results[:15]:  # Display first 15 results
    print(f"{src:<30} | {tgt:<20} | {custom_pred:<30} | {gpt2_pred:<30}")

# Save results for analysis
with open("evaluation_results.txt", "w") as f:
    for src, tgt, custom_pred, gpt2_pred in evaluation_results:
        f.write(f"{src}\t{tgt}\t{custom_pred}\t{gpt2_pred}\n")


Custom Model Device: cuda:0
GPT-2 Model Device: cuda:0
Loading dataset from: /home/lxp334/LLM_Final_Report/arithmetic__mixed.txt
Total samples from all datasets: 666666
Total Samples: 666666
Sample Data (Encoded): [(tensor([ 1,  5, 18, 10, 18, 14,  7, 18, 10, 18, 11,  1,  7, 15, 13,  1]), tensor([5]))]
Train Samples: 479998
Validation Samples: 53334
Test Samples: 133334


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Expression                     | Expected             | Custom LLM Prediction          | GPT-2 Prediction              
------------------------------------------------------------------------------------------------------------------------
     (-9)/12 + 0 - 38/(-8)     | 4                    | 555                            | 5                             
((-21)/210)/(-2 - (-8)/10)     | 1/12                 | 5555                           | 1/4                           
 ((-21)/9 - -1)/((-10)/(-15)). | -2                   | 5555                           | -2                            
(-2)/(-2) - 7*6/6              | -6                   | 5555                           | -5                            
  2*((-46)/76 - 2/(-19))       | -1                   | 5555                           | -1                            
 (10/(-6))/((-325)/(-195)).    | -1                   | 555                            | -1                            
  (4/(-21))/(432/504)          | -2/9  

## **Evaluation Metrics**

To evaluate the performance of both the Custom LLM and GPT-2 models, we compute two primary metrics:

### **1. Exact Match Accuracy**
- **Definition**:
  - Exact match accuracy measures the percentage of predictions that are exactly identical to the expected output.
  - This metric provides a strict evaluation of correctness.

- **Formula**:
  $$
  \text{Exact Match Accuracy (\%)} = \left( \frac{\text{Number of Exact Matches}}{\text{Total Predictions}} \right) \times 100
  $$

- **Purpose**:
  - Useful for scenarios where precision is critical, and partial matches are not acceptable.

---

### **2. Similarity Percentage Accuracy**
- **Definition**:
  - Similarity accuracy evaluates how close the model’s predictions are to the expected output, even if they are not an exact match.
  - This metric uses a similarity score (e.g., Levenshtein Ratio) to measure the degree of overlap.

- **Computation**:
  $$
  \text{Similarity Percentage (\%)} = (\text{Levenshtein Similarity Ratio}) \times 100
  $$

  - The average similarity across all predictions gives the overall Similarity Accuracy.

- **Purpose**:
  - This metric is useful for identifying cases where the model produces partially correct outputs due to formatting or minor rounding errors.
  
## **3. Error Rate**

### **Definition**
- Error Rate measures the percentage of predictions that differ from the expected results.
- It provides insight into the proportion of incorrect predictions made by the model.

### **Computation**
- Error Rate is inversely related to Exact Match Accuracy and is calculated as:

  $$
  \text{Error Rate (\%)} = 100 - \text{Exact Match Accuracy (\%)}
  $$

### **Purpose**
- **Quantifies Model Errors**:
  - Highlights the areas where the model struggles to produce the correct outputs.
- **Model Comparison**:
  - Helps compare models by showing their failure rates relative to their accuracy.
- **Improvement Opportunities**:
  - Identifies potential areas for optimization or retraining of the model.




In [14]:
from difflib import SequenceMatcher

# Function to calculate Exact Match Accuracy
def calculate_exact_match_accuracy(predictions, ground_truths):
    exact_matches = sum([1 for pred, tgt in zip(predictions, ground_truths) if pred == tgt])
    exact_match_accuracy = (exact_matches / len(ground_truths)) * 100
    return exact_match_accuracy

# Function to calculate Average Similarity Percentage
def calculate_similarity_accuracy(predictions, ground_truths):
    similarities = [
        SequenceMatcher(None, pred, tgt).ratio() * 100
        for pred, tgt in zip(predictions, ground_truths)
    ]
    average_similarity = sum(similarities) / len(similarities)
    return average_similarity

# Collect predictions and ground truths
custom_predictions = [result[2] for result in evaluation_results]  # Custom LLM Predictions
gpt2_predictions = [result[3] for result in evaluation_results]    # GPT-2 Predictions
ground_truths = [result[1] for result in evaluation_results]       # Expected Results

# Calculate metrics for both models
custom_exact_match = calculate_exact_match_accuracy(custom_predictions, ground_truths)
gpt2_exact_match = calculate_exact_match_accuracy(gpt2_predictions, ground_truths)

custom_similarity = calculate_similarity_accuracy(custom_predictions, ground_truths)
gpt2_similarity = calculate_similarity_accuracy(gpt2_predictions, ground_truths)

custom_error_rate = 100 - custom_exact_match
gpt2_error_rate = 100 - gpt2_exact_match

# Display metrics as a table
print(f"{'Model':<20} | {'Exact Match Accuracy (%)':<25} | {'Similarity Accuracy (%)':<25} | {'Error Rate (%)':<15}")
print("-" * 90)
print(f"{'Custom LLM':<20} | {custom_exact_match:<25.2f} | {custom_similarity:<25.2f} | {custom_error_rate:<15.2f}")
print(f"{'GPT-2':<20} | {gpt2_exact_match:<25.2f} | {gpt2_similarity:<25.2f} | {gpt2_error_rate:<15.2f}")


# Plotting metrics for comparison
import matplotlib.pyplot as plt
import numpy as np

# Data for plotting
models = ['Custom LLM', 'GPT-2']
exact_match = [custom_exact_match, gpt2_exact_match]
similarity = [custom_similarity, gpt2_similarity]
error_rate = [custom_error_rate, gpt2_error_rate]

x = np.arange(len(models))  # Label locations
width = 0.2  # Reduced width for thinner bars

fig, ax = plt.subplots(figsize=(8, 6))
bar1 = ax.bar(x - width, exact_match, width, label='Exact Match Accuracy', color='green')  # Exact Match: Green
bar2 = ax.bar(x, similarity, width, label='Similarity Accuracy', color='orange')          # Similarity: Orange
bar3 = ax.bar(x + width, error_rate, width, label='Error Rate', color='red')              # Error Rate: Red

# Add text labels to each bar
ax.bar_label(bar1, fmt='%.2f', padding=3)  # Add labels to Exact Match bars
ax.bar_label(bar2, fmt='%.2f', padding=3)  # Add labels to Similarity bars
ax.bar_label(bar3, fmt='%.2f', padding=3)  # Add labels to Error Rate bars

# Customize chart
ax.set_ylabel('Percentage (%)')
ax.set_title('Metrics Comparison Between Models')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.legend()

# Save the plot to a file
plt.tight_layout()
plt.savefig("accuracy_comparison_thin_bars.png", dpi=300)  # Save as a PNG file with high resolution
plt.close()  # Close the figure


Model                | Exact Match Accuracy (%)  | Similarity Accuracy (%)   | Error Rate (%) 
------------------------------------------------------------------------------------------
Custom LLM           | 0.00                      | 5.90                      | 100.00         
GPT-2                | 54.16                     | 75.58                     | 45.84          


## **Inference Time Analysis**

### **Definition**
- Inference time measures the time taken by a model to process a single input and generate a prediction.
- It reflects the computational efficiency and responsiveness of the model.

### **Computation**
- For each model, we calculate the average time taken to predict a result across all test inputs:
  $$
  \text{Average Inference Time} = \frac{\text{Total Time Taken for All Predictions}}{\text{Number of Predictions}}
  $$

### **Purpose**
  - Helps identify which model is faster during inference.
  - Useful for scenarios where real-time predictions are required or when computational resources are limited.
  - Faster inference times are desirable for production environments.


In [15]:
import time

def measure_inference_time(model, inputs, tokenizer):
    start_time = time.time()
    for input_text in inputs:
        # Handle ArithmeticTokenizer
        if isinstance(tokenizer, ArithmeticTokenizer):
            input_ids = torch.tensor([tokenizer.encode(input_text)], dtype=torch.long).to(device)
            # Custom LLM inference
            model(input_ids, input_ids[:, :-1])  # Call forward method
        else:
            # For Hugging Face tokenizer (GPT-2)
            input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
            # Add max_new_tokens to avoid max_length issues
            model.generate(input_ids, max_new_tokens=10)
    end_time = time.time()
    return (end_time - start_time) / len(inputs)  # Average time per prediction


import random

# Randomly sample 100 test expressions
sampled_test_expressions = random.sample(test_expressions, 500)

# Measure inference time on the subset for Custom LLM
custom_inference_time = measure_inference_time(custom_transformer_model, sampled_test_expressions, tokenizer)

# Measure inference time on the subset for GPT-2
gpt2_inference_time = measure_inference_time(gpt2_model, sampled_test_expressions, gpt2_tokenizer)

# Display inference times as a table
print(f"{'Model':<20} | {'Average Inference Time (seconds)':<30}")
print("-" * 55)
print(f"{'Custom LLM':<20} | {custom_inference_time:<30.5f}")
print(f"{'GPT-2':<20} | {gpt2_inference_time:<30.5f}")



NameError: name 'test_expressions' is not defined

## Error Analysis and Visualization for Model Predictions

### Key Steps:
1. **Error Categorization**:
   - Defined four error types: `Correct`, `Sign Errors`, `Rounding Issues`, and `Arithmetic Errors`.
   - Categorized model predictions by comparing them to ground truth.

2. **Pie Chart Visualization**:
   - Created pie charts for error distributions in both the `Custom LLM` and `GPT-2` models.

3. **Error Distribution Table**:
   - Printed a tabular summary of error counts and percentages for both models.

### Outputs:
- Visual and tabular representation of error distributions.
- Highlighted differences in error types between models.


In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import defaultdict

# Define an improved color palette
colors = ["#6D9EEB", "#F6B26B", "#FFE599", "#E06666"]  # Custom pastel colors

# Categorize errors for both models
def analyze_errors(results):
    error_types = {
        "Correct": 0,
        "Sign Errors": 0,
        "Rounding Issues": 0,
        "Arithmetic Errors": 0,
    }

    examples = defaultdict(list)  # Store examples for each error type

    for src, tgt, custom_pred, gpt2_pred in results:
        # ----- Custom LLM Errors -----
        if custom_pred == tgt:
            error_types["Correct"] += 1
        elif custom_pred.replace("-", "").replace("+", "") == tgt.replace("-", "").replace("+", ""):
            error_types["Sign Errors"] += 1
            examples["Sign Errors"].append((src, tgt, custom_pred))
        elif (
            custom_pred.replace(".", "", 1).isdigit() and tgt.replace(".", "", 1).isdigit()
            and abs(float(custom_pred) - float(tgt)) < 0.01
        ):
            error_types["Rounding Issues"] += 1
            examples["Rounding Issues"].append((src, tgt, custom_pred))
        else:
            error_types["Arithmetic Errors"] += 1
            examples["Arithmetic Errors"].append((src, tgt, custom_pred))

    return error_types, examples


# Perform error analysis
custom_errors, custom_examples = analyze_errors(evaluation_results)
gpt2_errors, gpt2_examples = analyze_errors([(src, tgt, gpt2_pred, gpt2_pred) for src, tgt, custom_pred, gpt2_pred in evaluation_results])

import matplotlib.pyplot as plt

# Define an improved color palette
colors = ["#F49595", "#A9CDD7", "#FDBA90", "#A99ABD"]  # Custom pastel colors

# Filter zero-value labels and counts
def filter_zero_values(labels, counts):
    filtered_labels = []
    filtered_counts = []
    for label, count in zip(labels, counts):
        if count > 0:  # Exclude zero-value counts
            filtered_labels.append(label)
            filtered_counts.append(count)
    return filtered_labels, filtered_counts

# Filtered data
custom_filtered_labels, custom_filtered_counts = filter_zero_values(error_labels, custom_error_counts)
gpt2_filtered_labels, gpt2_filtered_counts = filter_zero_values(error_labels, gpt2_error_counts)

# Filter zero-value labels and counts
def filter_zero_values(labels, counts):
    filtered_labels = []
    filtered_counts = []
    for label, count in zip(labels, counts):
        if count > 0:  # Exclude zero-value counts
            filtered_labels.append(label)
            filtered_counts.append(count)
    return filtered_labels, filtered_counts

# Filtered data
custom_filtered_labels, custom_filtered_counts = filter_zero_values(error_labels, custom_error_counts)
gpt2_filtered_labels, gpt2_filtered_counts = filter_zero_values(error_labels, gpt2_error_counts)

# Enhanced pie chart plotting with smaller sizes and label font adjustments
fig, axes = plt.subplots(1, 2, figsize=(7, 3.5))  # Reduced size by 30%

# Custom LLM pie chart
wedges1, texts1, autotexts1 = axes[0].pie(
    custom_filtered_counts,
    labels=custom_filtered_labels,
    autopct=lambda pct: f'{pct:.1f}%\n({int(round(pct * sum(custom_filtered_counts) / 100))})' if pct > 2 else '',
    startangle=0,
    colors=colors,
    explode=[0.03] * len(custom_filtered_labels),  # Slightly separated slices
    shadow=False,
    wedgeprops={'edgecolor': 'white', 'linewidth': 1}
)
axes[0].set_title("Custom LLM", fontsize=12, fontweight='bold', pad=15)

# Reduce label sizes for Custom LLM
for text in texts1:
    text.set_fontsize(5)  # Adjust font size for slice labels
for autotext in autotexts1:
    autotext.set_fontsize(5)  # Adjust font size for percentage labels

# GPT-2 pie chart
wedges2, texts2, autotexts2 = axes[1].pie(
    gpt2_filtered_counts,
    labels=gpt2_filtered_labels,
    autopct=lambda pct: f'{pct:.1f}%\n({int(round(pct * sum(gpt2_filtered_counts) / 100))})' if pct > 2 else '',
    startangle=0,
    colors=colors,
    explode=[0.03] * len(gpt2_filtered_labels),  # Slightly separated slices
    shadow=False,
    wedgeprops={'edgecolor': 'white', 'linewidth': 1}
)
axes[1].set_title("Fine-Tuned GPT-2", fontsize=12, fontweight='bold', pad=15)

# Reduce label sizes for GPT-2
for text in texts2:
    text.set_fontsize(5)  # Adjust font size for slice labels
for autotext in autotexts2:
    autotext.set_fontsize(5)  # Adjust font size for percentage labels

# Adjust layout for better spacing
plt.tight_layout()
plt.subplots_adjust(top=0.8)  # Add more space between the chart and the top edge

# Save the improved chart
plt.savefig("Error_analysis_pie_charts.png", dpi=300)
plt.close()


# Print error percentages as a table
def print_error_table(model_name, error_counts, total_count):
    print(f"\n{model_name} Error Distribution:")
    print(f"{'Error Type':<20} | {'Count':<10} | {'Percentage':<10}")
    print("-" * 45)
    for i, count in enumerate(error_counts):
        percentage = (count / total_count) * 100
        print(f"{error_labels[i]:<20} | {count:<10} | {percentage:.2f}%")
    print("-" * 45)

# Print tables for both models
print_error_table("Custom LLM", custom_error_counts, sum(custom_error_counts))
print_error_table("GPT-2", gpt2_error_counts, sum(gpt2_error_counts))

